In [2]:
import pandas as pd

In [3]:
weather_df = pd.read_csv("Rf data.csv").dropna()
weather_df = weather_df.rename(columns={'station_name':'District','yyyy':'Year'})
columns = ['District','Year','1','2','3','4','5','6','7','8','9','10','11','12']
weather_df = weather_df[columns]
weather_df['District'] = weather_df['District'].str.capitalize()
weather_df.loc[weather_df['District']=='Katugastota','District']= 'Kandy'
weather_df.loc[weather_df['District']=='Pottuvil','District']= 'Ampara'
weather_df.loc[weather_df['District']=='Nuwara eliya','District']= 'Nuwara Eliya'
weather_df['Year']=weather_df['Year'].astype(int)

disaster_df = pd.read_csv('Preprocessed.csv').dropna(subset=['District'])

In [4]:
rainfall_melted1 = pd.melt(weather_df, id_vars=[ 'District', 'Year'], var_name='Month', value_name='Rainfall(mm)')

disaster_df['Rainfall(mm)'] = 0
disaster_df['Wind Speed(mph)'] = 0

In [5]:
rainfall_melted1['Month'] = rainfall_melted1['Month'].astype(int)
for idx, row in disaster_df.iterrows():
    
    matching_rows = rainfall_melted1[(rainfall_melted1['Year'] == row['Year']) &
                                    (rainfall_melted1['Month'] == row['Month']) &
                                    (rainfall_melted1['District'] == row['District'])]
    
    if not matching_rows.empty:
        matching_index = matching_rows.index[0]
        rainfall_value = matching_rows.at[matching_index, 'Rainfall(mm)']
        disaster_df.at[idx, 'Rainfall(mm)'] = rainfall_value


In [6]:
avg_wind = pd.read_csv('Wind_Average.csv')
avg_rainfall = pd.read_csv('Average RF.csv')

avg_rainfall_melted = pd.melt(avg_rainfall, id_vars=[ 'District'], var_name='Month', value_name='Rainfall(mm)')
wind_melted = pd.melt(avg_wind, id_vars=[ 'District'], var_name='Month', value_name='Wind Speed(mph)')

In [7]:
avg_rainfall_melted = avg_rainfall_melted.reset_index(drop=True)

for idx, row in disaster_df.iterrows():
    matching_row_rf = avg_rainfall_melted[(avg_rainfall_melted['Month'].astype(int) == row['Month']) &
                                    (avg_rainfall_melted['District'] == row['District'])]
    matching_row_ws = wind_melted[(wind_melted['Month'].astype(int) == row['Month']) &
                                    (wind_melted['District'] == row['District'])]
    
    if not matching_row_rf.empty and row['Rainfall(mm)']==0.0:
        new_rainfall_value = matching_row_rf.iloc[0]['Rainfall(mm)']
        disaster_df.at[idx, 'Rainfall(mm)'] = new_rainfall_value
    if not matching_row_ws.empty:
        matching_index = matching_row_ws.index[0]
        wind_value = matching_row_ws.at[matching_index, 'Wind Speed(mph)']
        disaster_df.at[idx, 'Wind Speed(mph)'] = wind_value


disaster_df.to_csv('Weather Inserted.csv',index=False)
        